# Preparing National Election Data for the House of Representatives

The official source keeps all data - including primary elections - in a single file. The purpose of this notebook is to clean the file so that only the general election is represented and all rows that do not include general election results are removed. Such rows could be state headers, district summaries and candidates that were eliminated in the primaries.

In [1]:
# Import libraries used in this notebook
import requests, os

from statistics import mean

import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

pd.set_option('display.max_rows', 2000)

import xlrd

In [2]:
#Local file path
raw_data = '../../raw-data/federal-election-results/federalelections2016.xlsx'

#Test if file already exist. Change replace to True to overwrite local file
replace = False
if os.path.isfile(raw_data) or replace:
    print("File already exists")
else:
    print("Downloading file")
    #Get the file from the Federal Election Committe's website
    r = requests.get('https://transition.fec.gov/pubrec/fe2016/federalelections2016.xlsx')

    #Save it to disk
    with open(raw_data,'wb') as f: 
        f.write(r.content) 

File already exists


In [3]:
#Load the sheet with 
house_US = pd.read_excel(raw_data, sheet_name='2016 US House Results by State')
#Rename column names with spaces or other characters now suitable for code
house_US.rename(columns={1:'row_excel','CANDIDATE NAME':'CANDIDATE_NAME','TOTAL VOTES':'TOTAL_VOTES',
                            'PRIMARY VOTES':'PRIMARY_VOTES','GENERAL VOTES ':'GENERAL_VOTES','FEC ID#':'FEC_ID',
                            'STATE ABBREVIATION':'STATE_ABB','RUNOFF VOTES':'RUNOFF_VOTES',
                            'GE RUNOFF ELECTION VOTES (LA)':'RUNOFF_VOTES_LA'}, inplace=True)
list(house_US.columns.values)

['row_excel',
 'STATE_ABB',
 'STATE',
 'D',
 'FEC_ID',
 '(I)',
 'CANDIDATE NAME (First)',
 'CANDIDATE NAME (Last)',
 'CANDIDATE_NAME',
 'TOTAL_VOTES',
 'PARTY',
 'PRIMARY_VOTES',
 'PRIMARY %',
 'RUNOFF_VOTES',
 'RUNOFF %',
 'GENERAL_VOTES',
 'GENERAL %',
 'RUNOFF_VOTES_LA',
 'GE RUNOFF ELECTION % (LA)',
 'COMBINED GE PARTY TOTALS (CT, NY, SC)',
 'COMBINED % (CT, NY, SC)',
 'GE WINNER INDICATOR',
 'FOOTNOTES']

## General prep
Start by trimming excessivle spaces from the FEC's Excel file.

In [4]:
#Remove excessive spaces before and after ID that makes the same ID differnt when 
# read by the code, for example "FL_24" and "FL_24 "
house_US.D = house_US.D.str.strip()

#Remove excess spaces for party abbreviations
house_US.PARTY = house_US.PARTY.str.strip()

Create a nation wide District ID

In [5]:
#Drop rows with missing value for district or state
house_US = house_US.dropna(subset=['D','STATE'])

#This is just correcting a typo in KY district 1. Must correct it here as
#we will merge on district ID before we clean state specific rows
house_US.loc[house_US.D == "1 - UNEXPIRED TERM", 'D'] = "01 - UNEXPIRED TERM"

# Create nationally unique ID
house_US["D_ID"] = house_US.STATE_ABB + "_" + house_US.D.map(str)

Copy aggregation rows to columns. State totals and district totals

In [6]:
#Subset district total to a df to be merged back to all rows
votes_dist_agg = house_US.loc[house_US.TOTAL_VOTES == "District Votes:"]
votes_dist_agg = votes_dist_agg[['D_ID','GENERAL_VOTES']]
votes_dist_agg.columns = ['D_ID','DISTRICT_VOTES']

#Subset state total to a df to be merged back to all rows
votes_state_agg = house_US.loc[house_US.TOTAL_VOTES == "Total State Votes:"]
votes_state_agg = votes_state_agg[['STATE_ABB','GENERAL_VOTES']]
votes_state_agg.columns = ['STATE_ABB','STATE_VOTES']

In [7]:
#Delete aggregation rows
house_US = house_US.loc[house_US.TOTAL_VOTES != "Party Votes:"]
house_US = house_US.loc[house_US.TOTAL_VOTES != "District Votes:"]
house_US = house_US.loc[house_US.TOTAL_VOTES != "Total State Votes:"]

#Keep only variables that we need
house_US = house_US[["STATE_ABB",'STATE','D_ID',"D",'CANDIDATE_NAME','PARTY'
                     ,'PRIMARY_VOTES','GENERAL_VOTES','RUNOFF_VOTES','RUNOFF_VOTES_LA']]

#Merge back teh aggregates we are interested in
house_US = pd.merge(house_US, votes_dist_agg, on='D_ID', how='outer')
house_US = pd.merge(house_US, votes_state_agg, on='STATE_ABB')

#Look at the data at this point
house_US.head(8)

,STATE_ABB,STATE,D_ID,D,CANDIDATE_NAME,PARTY,PRIMARY_VOTES,GENERAL_VOTES,RUNOFF_VOTES,RUNOFF_VOTES_LA,DISTRICT_VOTES,STATE_VOTES
0,AL,Alabama,AL_01,01,"Byrne, Bradley",R,71310,208083,NaN,NaN,215893,1889685
1,AL,Alabama,AL_01,01,"Young, Dean",R,47319,NaN,NaN,NaN,215893,1889685
2,AL,Alabama,AL_01,01,Scattered,W,NaN,7810,NaN,NaN,215893,1889685
3,AL,Alabama,AL_02,02,"Roby, Martha",R,78689,134886,NaN,NaN,276584,1889685
4,AL,Alabama,AL_02,02,"Gerritson, Becky",R,33015,NaN,NaN,NaN,276584,1889685
5,AL,Alabama,AL_02,02,"Rogers, Robert L. ""Bob""",R,6856,NaN,NaN,NaN,276584,1889685
6,AL,Alabama,AL_02,02,"Mathis, Nathan",D,Unopposed,112089,NaN,NaN,276584,1889685
7,AL,Alabama,AL_02,02,Scattered,W,NaN,29609,NaN,NaN,276584,1889685


# State specific cleaning

In [8]:
#Function that make sure all dfs are saved identically
def state_csv_save(df):
    df = df[["STATE_ABB",'STATE','D_ID','CANDIDATE_NAME','PARTY','VOTES','RUNOFF_VOTES',"DISTRICT_VOTES","STATE_VOTES"]]
    state = df.STATE_ABB.unique()[0]
    df.to_csv('../by-state/house-' + state + '-cleaned.csv', index=False)  
    return df

### Alabama

In [9]:
state = "AL"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'W' 'D']


### Alaska

In [10]:
state = "AK"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB' 'NAF' 'W']


### Arizona

In [11]:
state = "AZ"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(GRE)/GRE", 'PARTY'] = "GRE"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'GRE' 'W']


### Arkansas

In [12]:
state = "AR"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'LIB' 'D' 'W']


### California

In [13]:
state = "CA"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()

house_state = house_state.dropna(subset=['CANDIDATE_NAME'])

house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(R)/R", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "W(GRE)", 'PARTY'] = "GRE"
house_state.loc[house_state.PARTY == "W(NOP)", 'PARTY'] = "NOP"

house_state = house_state.rename(columns={'PRIMARY_VOTES':'VOTES'})

house_state['RUNOFF_VOTES'] = house_state['GENERAL_VOTES']

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'NOP' 'LIB' 'GRE' 'PAF']


### Colorado

In [14]:
state = "CO"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'LIB' 'W' 'GRE']


### Cincinatti

In [15]:
state = "CT"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "R/W", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'WF' 'R' 'GRE' 'W' 'LIB' 'IP']


### Delaware

In [16]:
state = "DE"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'GRE' 'LIB' 'W']


### District of Columbia

In [17]:
state = "DC"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(DCG)", 'PARTY'] = "DCG"
house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'LIB' 'DCG' 'W']


### Florida

In [18]:
state = "FL"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

#In FL_24 one D candidate ran un-opposed. Give that candidate average vote for all candidates in all FL districts
all_general_votes = house_state.loc[:,"GENERAL_VOTES"]
all_general_votes = all_general_votes[(all_general_votes != np.nan) & (all_general_votes != 'Unopposed')]
mean_general_vote = int(np.mean(all_general_votes))
house_state.loc[house_state.GENERAL_VOTES == "Unopposed", 'GENERAL_VOTES'] = mean_general_vote

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())

#house_state.head(1000)

['R' 'D' 'LBF' 'W' 'NPA']


### Georgia

In [19]:
state = "GA"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.D_ID == "GA_03", 'RUNOFF_VOTES'] = np.nan

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'W' 'D']


### Hawaii

District 1 had two elections has the incumbent representative had deceased. The "Unexpired term" was the election for the reminder of the deceeased representatives term. The "Full term" Election is for the next term. See more [here](https://en.wikipedia.org/wiki/2016_Hawaii%27s_1st_congressional_district_special_election).

In [20]:
state = "HI"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state[house_state.D != "01 - UNEXPIRED TERM"]

house_state.loc[house_state.D == "01 - FULL TERM", 'D_ID'] = "HI_01"
house_state.loc[house_state.D == "01 - FULL TERM", 'D'] = "01"

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'LIB' 'N']


### Idaho

In [21]:
state = "ID"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'W' 'CON']


## Illinois

In [22]:
state = "IL"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

#house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'W' 'GRE']


### Indiana

In [23]:
state = "IN"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'LIB' 'W' 'R']


### Iowa

In [24]:
state = "IA"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'W' 'LIB' 'NNE']


### Kansas

In [25]:
state = "KS"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "R/W", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'IND' 'LIB' 'D']


### Kentucky
https://en.wikipedia.org/wiki/2016_United_States_House_of_Representatives_elections_in_Kentucky#Special_election

In [26]:
state = "KY"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = house_state[house_state.D != "01 - UNEXPIRED TERM"]

house_state.loc[house_state.D == "01 - FULL TERM", 'D_ID'] = "KY_01"
house_state.loc[house_state.D == "01 - FULL TERM", 'D'] = "01"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'W']


### Luisiana

In [27]:
state = "LA"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state['RUNOFF_VOTES'] = house_state['RUNOFF_VOTES_LA']

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB' 'GRE' 'NPA' 'OTH']


### Maine

In [28]:
state = "ME"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(LIB)", 'PARTY'] = "LIB"
house_state.loc[house_state.PARTY == "W(IND)", 'PARTY'] = "IND"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'LIB' 'IND']


### Maryland

In [29]:
state = "MD"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB' 'W' 'GRE']


### Massachusetts

In [30]:
state = "MA"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'IND' 'LIB' 'W' 'R' 'U']


### Michigan`

In [31]:
state = "MI"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(D)/D", 'PARTY'] = "D"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB' 'GRE' 'UST' 'W' 'NLP' 'NPA' 'WC']


### Minnesota

In [32]:
state = "MN"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['DFL' 'R' 'W' 'IDP' 'LMN']


### Mississippi

In [33]:
state = "MS"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())

#house_state.head(1000)

['R' 'D' 'LIB' 'REF' 'IND' 'VPA']


### Missouri

In [34]:
state = "MO"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'LIB' 'GRE' 'CON' 'W']


### Montana

In [35]:
state = "MT"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB']


### Nebraska

In [36]:
state = "NE"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB']


### Nevada

In [37]:
state = "NV"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'NPY' 'IAP' 'LIB']


### New Hampshire

In [38]:
state = "NH"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "R/W", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'IND' 'LIB' 'W']


### New Jersey

In [39]:
state = "NJ"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'WDB' 'LIB' 'AO' 'MGW' 'RNN' 'PIP' 'FPR' 'CON' 'EG' 'GRE' 'WUA'
 'NSA' 'WOP' 'NBP' 'FI' 'LMP' 'TED' 'WTP']


### New Mexico

In [40]:
state = "NM"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'W']


### New York

In [41]:
state = "NY"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "R/TRP", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'CRV' 'REF' 'IDP' 'D' 'WEP' 'WF' 'W' 'GRE' 'BLM' 'HBP']


### North Carolina

In [42]:
state = "NC"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'LIB']


### North Dakota

In [43]:
state = "ND"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "W(DNL)", 'PARTY'] = "DNL"
house_state.loc[house_state.PARTY == "W(LIB)", 'PARTY'] = "LIB"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'DNL' 'LIB' 'W']


### Ohio

In [44]:
state = "OH"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'W' 'N' 'GRE']


### Oklahoma

Oklahoma district 5 had a run-off in the Democratic Party's primary.

In [45]:
state = "OK"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

#In OK_01 one R candidate ran un-opposed. Give that candidate average vote for all candidates in all FL districts
all_general_votes = house_state.loc[:,"GENERAL_VOTES"]
all_general_votes = all_general_votes[(all_general_votes != np.nan) & (all_general_votes != 'Unopposed')]
mean_general_vote = int(np.mean(all_general_votes))
house_state.loc[house_state.GENERAL_VOTES == "Unopposed", 'GENERAL_VOTES'] = mean_general_vote

house_state['RUNOFF_VOTES'] = np.nan

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'IND' 'LIB']


### Oregon

In [46]:
state = "OR"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "D/IP", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "D/PRO/WF/IP", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"

house_state.loc[house_state.PARTY == "IP/R", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "R/CON", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "R/IP", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

house_state.loc[house_state.PARTY == "W(IP)", 'PARTY'] = "IP"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'LIB' 'W' 'PRO' 'PG']


### Pennsylvania

Special election in district 2 https://en.wikipedia.org/wiki/2016_United_States_House_of_Representatives_elections_in_Pennsylvania#District_2

In [47]:
state = "PA"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state[house_state.D != "02 - UNEXPIRED TERM"]

house_state.loc[house_state.D == "02 - FULL TERM", 'D_ID'] = "PA_02"
house_state.loc[house_state.D == "02 - FULL TERM", 'D'] = "02"

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(D)/D", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(D)/W", 'PARTY'] = "D"

house_state.loc[house_state.PARTY == "W(R)/R", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'W' 'LIB']


### Rhode Island

In [48]:
state = "RI"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R' 'W' 'IND']


### South Carolina

In [49]:
state = "SC"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'WF' 'GRE' 'AM' 'LIB' 'W' 'CON']


### South Dakota

In [50]:
state = "SD"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D']


### Tennesse

In [51]:
state = "TN"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'IND' 'W']


### Texas

Runoff in primary District 15 R & D, District 18 R, District 19 R

In [52]:
state = "TX"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state['RUNOFF_VOTES'] = np.nan

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB' 'W' 'GRE']


### Utah

In [53]:
state = "UT"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB' 'UN' 'CON']


### Vermont

In [54]:
state = "VT"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(PRO)", 'PARTY'] = "PRO"
house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

house_state.loc[house_state.PARTY == "D/R", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'LBU' 'W']


### Virginia

In [55]:
state = "VA"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'IND' 'W']


### Washington

In [56]:
state = "WA"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['D' 'R']


### West Virginia

In [57]:
state = "WV"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'LIB']


### Wisconsin

In [58]:
state = "WI"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(LIB)", 'PARTY'] = "LIB"
house_state.loc[house_state.PARTY == "W(CON)", 'PARTY'] = "CON"
house_state.loc[house_state.PARTY == "W(WG)", 'PARTY'] = "WG"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'TC' 'LIB' 'W' 'IND']


### Wyoming

In [59]:
state = "WY"

house_state = house_US.loc[house_US.STATE_ABB==state].copy()
house_state = house_state.dropna(subset=['PARTY'])

house_state = house_state.rename(columns={'GENERAL_VOTES':'VOTES'}).dropna(subset=['VOTES'])

house_state.loc[house_state.PARTY == "W(R)", 'PARTY'] = "R"
house_state.loc[house_state.PARTY == "W(D)", 'PARTY'] = "D"
house_state.loc[house_state.PARTY == "W(LIB)", 'PARTY'] = "LIB"
house_state.loc[house_state.PARTY == "W(CON)", 'PARTY'] = "CON"

house_state = state_csv_save(house_state)

print(house_state.PARTY.unique())
#house_state.head(1000)

['R' 'D' 'CON' 'LIB' 'W']


# Append all cleaned State files to one csv file

In [60]:
house_us_cleaned = pd.DataFrame()

state_files = os.listdir('../by-state')
for state_file in state_files:
    house_state_cleaned = pd.read_csv('../by-state/' + state_file)
    house_us_cleaned = house_us_cleaned.append(house_state_cleaned, ignore_index = True) 

house_us_cleaned.to_csv('../house-by-state-cleaned.csv', index=False)
house_us_cleaned.head(25)

,STATE_ABB,STATE,D_ID,CANDIDATE_NAME,PARTY,VOTES,RUNOFF_VOTES,DISTRICT_VOTES,STATE_VOTES
0,AK,Alaska,AK_00,"Young, Don",R,155088,NaN,308198.0,308198
1,AK,Alaska,AK_00,"Lindbeck, Steve",D,111019,NaN,308198.0,308198
2,AK,Alaska,AK_00,"McDermott, Jim C.",LIB,31770,NaN,308198.0,308198
3,AK,Alaska,AK_00,"Souphanavong, Bernie",NAF,9093,NaN,308198.0,308198
4,AK,Alaska,AK_00,Scattered,W,1228,NaN,308198.0,308198
5,AL,Alabama,AL_01,"Byrne, Bradley",R,208083,NaN,215893.0,1889685
6,AL,Alabama,AL_01,Scattered,W,7810,NaN,215893.0,1889685
7,AL,Alabama,AL_02,"Roby, Martha",R,134886,NaN,276584.0,1889685
8,AL,Alabama,AL_02,"Mathis, Nathan",D,112089,NaN,276584.0,1889685
9,AL,Alabama,AL_02,Scattered,W,29609,NaN,276584.0,1889685
